In [16]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
import zipfile
import io
import xlsxwriter
import time

In [84]:
ano = int(input())
mes = int(input())
ano = str(ano)
mes = str(mes).zfill(2)

In [88]:
mes

'02'

In [86]:
#Funções:

###Função requisicao dados CVN
# Função para fazer a requisição e tratar os possíveis erros
def fazer_requisicao(url):
    try:
        # Realizar a requisição
        response = requests.get(url)

        # Verificar o status code da resposta
        if response.status_code == 200:
            print('Requisição bem-sucedida para '+mes+'/'+ano+'.')
            return response
        elif response.status_code == 404:
            print('Página não encontrada (erro 404).')
            return  # Interrompe o código aqui se o status code for 404
        else:
            print(f'Erro inesperado: Status code {response.status_code}')
            return  # Interrompe o código em caso de status code diferente de 200 e 404

    except requests.ConnectionError:
        print('Erro de conexão: Verifique sua conexão de internet.')
    except requests.Timeout:
        print('Erro de timeout: A requisição excedeu o tempo limite.')
    except requests.RequestException as e:
        print(f'Erro na requisição: {e}')

In [93]:
link1 = 'https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_'+ano+mes+'.zip'

requisicao_inf_diario = fazer_requisicao(link1)

zf_inf_diario = zipfile.ZipFile(io.BytesIO(requisicao_inf_diario.content))

arquivo_inf_diario = 'inf_diario_fi_'+ano+mes+'.csv'

zf_inf_diario = zf_inf_diario.open(arquivo_inf_diario)

lines_inf_diario = zf_inf_diario.readlines()

lines_inf_diario = [i.strip().decode('ISO-8859-1') for i in lines_inf_diario]

lines_inf_diario = [i.split(';') for i in lines_inf_diario]
      
inf_dia = pd.DataFrame(lines_inf_diario[1:], columns = lines_inf_diario[0])

Requisição bem-sucedida para 02/2024.


In [94]:
inf_dia

,TP_FUNDO,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
0,FI,00.017.024/0001-53,2024-02-01,1143827.67,34.571524200000,1145999.98,0.00,0.00,1
1,FI,00.017.024/0001-53,2024-02-02,1144304.09,34.582609700000,1146367.45,0.00,0.00,1
2,FI,00.017.024/0001-53,2024-02-05,1144776.75,34.594115200000,1146748.84,0.00,0.00,1
3,FI,00.017.024/0001-53,2024-02-06,1145251.90,34.605507200000,1147126.47,0.00,0.00,1
4,FI,00.017.024/0001-53,2024-02-07,1145392.84,34.616973700000,1147506.57,0.00,0.00,1
...,...,...,...,...,...,...,...,...,...
490971,FI,97.929.213/0001-34,2024-02-23,86573988.67,11.703107280000,84123516.06,0.00,0.00,2
490972,FI,97.929.213/0001-34,2024-02-26,86570206.49,11.703141250000,84123760.25,0.00,0.00,2
490973,FI,97.929.213/0001-34,2024-02-27,86726281.24,11.724777760000,84279286.39,0.00,0.00,2
490974,FI,97.929.213/0001-34,2024-02-28,85599197.62,11.728172290000,84303686.70,0.00,0.00,2


In [95]:
link2 = 'https://dados.cvm.gov.br/dados/FI/DOC/LAMINA/DADOS/lamina_fi_'+ano+mes+'.zip'

requisicao_lamina = fazer_requisicao(link2)


zf_lamina = zipfile.ZipFile(io.BytesIO(requisicao_lamina.content))

arquivo_lamina = 'lamina_fi_'+ano+mes+'.csv'

zf_lamina = zf_lamina.open(arquivo_lamina)

lines_lamina = zf_lamina.readlines()

lines_lamina = [i.strip().decode('ISO-8859-1') for i in lines_lamina]

lines_lamina = [i.split(';') for i in lines_lamina]

lamina_mes = pd.DataFrame(lines_lamina[1:], columns = lines_lamina[0])

Requisição bem-sucedida para 02/2024.


In [97]:
lamina_mes.columns

Index(['CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_COMPTC', 'NM_FANTASIA',
       'ENDER_ELETRONICO', 'PUBLICO_ALVO', 'RESTR_INVEST', 'OBJETIVO',
       'POLIT_INVEST', 'PR_PL_ATIVO_EXTERIOR', 'PR_PL_ATIVO_CRED_PRIV',
       'PR_PL_ALAVANC', 'PR_ATIVO_EMISSOR', 'DERIV_PROTECAO_CARTEIRA',
       'RISCO_PERDA', 'RISCO_PERDA_NEGATIVO', 'PR_PL_APLIC_MAX_FUNDO_UNICO',
       'INVEST_INICIAL_MIN', 'INVEST_ADIC', 'RESGATE_MIN',
       'HORA_APLIC_RESGATE', 'VL_MIN_PERMAN', 'QT_DIA_CAREN', 'CONDIC_CAREN',
       'CONVERSAO_COTA_COMPRA', 'QT_DIA_CONVERSAO_COTA_COMPRA',
       'CONVERSAO_COTA_CANC', 'QT_DIA_CONVERSAO_COTA_RESGATE',
       'TP_DIA_PAGTO_RESGATE', 'QT_DIA_PAGTO_RESGATE', 'TP_TAXA_ADM',
       'TAXA_ADM', 'TAXA_ADM_MIN', 'TAXA_ADM_MAX', 'TAXA_ADM_OBS', 'TAXA_ENTR',
       'CONDIC_ENTR', 'QT_DIA_SAIDA', 'TAXA_SAIDA', 'CONDIC_SAIDA',
       'TAXA_PERFM', 'PR_PL_DESPESA', 'DT_INI_DESPESA', 'DT_FIM_DESPESA',
       'ENDER_ELETRONICO_DESPESA', 'VL_PATRIM_LIQ', 'CLASSE_RISCO_ADMIN',
       'PR_REN

In [58]:
link
https://dados.cvm.gov.br/dados/FI/DOC/LAMINA/DADOS/lamina_fi_202402.zip

'https://dados.cvm.gov.br/dados/FI/DOC/LAMINA/DADOS/202402.zip'

In [77]:
zipfile.ZipFile(io.BytesIO(r.content))

<zipfile.ZipFile file=<_io.BytesIO object at 0x0000019105344E00> mode='r'>